In [2]:
%reload_ext autoreload
%autoreload 2

from ripple_heterogeneity.utils import (
    functions,
    loading,
    compress_repeated_epochs,
    batch_analysis,
    add_new_deep_sup,
    custom_plots
)
from ripple_heterogeneity.replay import replay_run, replay_fig, replay_participation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import pickle
import seaborn as sns
import nelpy as nel
import nelpy.plotting as npl
import random
from matplotlib.ticker import AutoMinorLocator

functions.set_plotting_defaults()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
fig_save_path = r'D:\github\ad_ied\results'


In [3]:
df = pd.DataFrame()
df["basepath"] = [
    r"X:\AD_sessions\AD_10_sess25\AZ10_210806_sess25",
    r"X:\AD_sessions\AD_10_sess25\AZ15_221102_sess8",
    r"X:\AD_sessions\AD_10_sess25\Naz1_210324_sess5",
    r"X:\AD_sessions\WT_1_sess26\Naz1_210811_sess26"
]